# Imports

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import vectorbtpro as vbt
import numpy as np


/Users/ericervin/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


# Initialization

In [2]:
vbt.settings.wrapping ["freq"]                  = "1m"
vbt.settings.portfolio['init_cash']             = 10000
vbt.settings.set_theme("dark")
vbt.settings.plotting["layout"]["width"]        = 800
vbt.settings.plotting['layout']['height']       = 300

# Settings and Parameters

In [3]:
pickle_files_path = "../data/RID0029_LSTM_pw38_lb250_bt2000_mem6000/*.pkl"
prediction_window = 38
filename_prefix = pickle_files_path.split('/')[-2]
min_num_entries          = 100
excel_output_file_name  = f"../results/{filename_prefix + '.xlsx'}"
lstm_output_file_name   = f"../results/{filename_prefix + 'lstm_model_outputs.csv'}"

# Local Imports

In [4]:
import os
import sys

sys.path.append(os.getcwd())

In [5]:
from lstm_analysis_utils import (read_pickle_files_into_df, add_forward_prices_to_df, generate_fwd_actual_column
                                 , generate_df_with_euclidean_distances, calculate_slopes, calculate_correlation_slopes
                                )
from lstm_results_utils import (export_results, store_backtest_results)
from lstm_only_backtests import run_backtest_lstm_recommendations_reversal_exits, run_backtests_lstm_recommendations_prediction_size_exit
from lstm_analysis_constants import EntryType
from quantile_value import generate_quantile_bands, extract_boundary_values_from_quantile_bands
from prediction_window_slopes import PredictionWindowSlopes
from long_slope_short_slope_backtests import run_backtest_long_slope_short_slope_prediction_size_exit, run_backtest_long_slope_short_slope_fractional_exits
from long_minus_short_backtests import run_backtest_long_minus_short_entry_type_long_only, run_backtest_long_minus_short_entry_type_short_only, run_backtest_long_minus_short_entry_type_long_short

# Processing

In [6]:
def process_LSTM_model_results(pickle_files_path, prediction_window):
    df = read_pickle_files_into_df(pickle_files_path)
    add_forward_prices_to_df(df, prediction_window)
    df= df.copy()
    generate_fwd_actual_column(df)
    df= generate_df_with_euclidean_distances(df, prediction_window)
    calculate_slopes(df)
    calculate_correlation_slopes(df)
    df.index = pd.to_datetime(df['close_time'], utc=True, unit='s')
    return df

# df = process_LSTM_model_results(pickle_files_path, prediction_window)
# df.to_csv(lstm_output_file_name)
df = pd.read_csv(lstm_output_file_name)
df.to_csv('RID0029_LSTM_pw38_lb250_bt2000_mem6000.csv')

# Running backtest

In [7]:
min_long_slope  = df["long_slope"].min()
max_long_slope  = df["long_slope"].max()
min_short_slope = df["short_slope"].min()
max_short_slope = df["short_slope"].max()

In [8]:
num_quantiles               = 5
threshold_increment         = 0.001
quantiles                   = np.linspace(0, 1, num=num_quantiles + 1)

# Paramater Combinations in VBT

In [9]:
def lms_slope_type_3(long_minus_short, long_slope, short_slope, lms_threshold, long_slope_thresh, short_slope_thresh):
    entries       = pd.Series(np.where((long_minus_short < lms_threshold) & (long_slope > long_slope_thresh), True, False))
    short_entries = pd.Series(np.where((long_minus_short < lms_threshold) & (short_slope < short_slope_thresh), True, False))
    
    return entries, short_entries

# Create an indicator factory
lms_slope_type_3_indicator = vbt.IndicatorFactory(
    class_name  ='LongMinusShortSlopeType3', # name of the class
    short_name  ='lmsSlope3', # name of the indicator
    input_names =['long_minus_short', 'long_slope', 'short_slope'], # names of input arguments
    param_names =['lms_threshold', 'long_slope_thresh', 'short_slope_thresh'], # names of parameters
    output_names=['entries', 'short_entries'], # names of output values
).with_apply_func(
    lms_slope_type_3, # function to apply
    takes_1d=True, # whether the function takes 1-dim. arrays as input
    lms_threshold=0.5, # default value for parameter 'lms_threshold'
    long_slope_thresh=0.0, # default value for parameter 'long_slope_thresh'
    short_slope_thresh=0.0, # default value for parameter 'short_slope_thresh'
)
  

You created the strategy up above but you haven't run it yet. In this next cell we run it with a lot of different combinations. This basically builds a big matrix of all the different strategy combinations with each having a different `lms_threshold` and `long_slope_thresh` and `short_slope_thresh`. We will use these to simulate a portfolio after this.

In [10]:
num_increments         = 30

lms_min                 = 0.6 #df.long_minus_short.min()
lms_max                 = 1 #df.long_minus_short.max()
long_slope_min          = 0 #df.long_slope.min()
long_slope_max          = df.long_slope.max()
short_slope_min         = df.short_slope.min()
short_slope_max         = df.short_slope.max()
lms_increment           = abs((df.long_minus_short.max()-df.long_minus_short.min())/num_increments)
long_slope_increment    = abs((df.long_slope.max()-df.long_slope.min())/num_increments)
short_slope_increment   = abs((df.short_slope.max()-df.short_slope.min())/num_increments)

In [11]:
lms_strategy = lms_slope_type_3_indicator.run(
    long_minus_short    =df['long_minus_short'],
    long_slope          =df['long_slope'],
    short_slope         =df['short_slope'],
    lms_threshold       =np.arange(lms_min, lms_max, lms_increment),
    long_slope_thresh   =np.arange(long_slope_min, long_slope_max, long_slope_increment),
    short_slope_thresh  =np.arange(short_slope_min, short_slope_max, short_slope_increment),
    param_product=True, # True: all combinations of parameters, False: only one combination for each parameter
)

Now let's run a portfolio simulation on all of those different parameter combinations. Note, the first time you run this it might take a bit but as you play and run it again it will get really fast.

In [12]:
multiple_pf = vbt.Portfolio.from_signals(
    close               =df['BTCUSDT_Close'],
    high                =df['BTCUSDT_High'],
    low                 =df['BTCUSDT_Low'],
    open                =df['BTCUSDT_Open'],
    entries             =lms_strategy.entries,
    short_entries       =lms_strategy.short_entries,
    td_stop             =prediction_window,
    time_delta_format   ='Rows',
    accumulate          =False,
    
)

print(multiple_pf.stats()) # Prints the average of all of the simulations

Start                                                  0
End                                                66201
Period                                  45 days 23:22:00
Start Value                                      10000.0
Min Value                                    7216.927041
Max Value                                   17736.783369
End Value                                    9295.434392
Total Return [%]                               -7.045656
Benchmark Return [%]                          -26.830306
Total Time Exposure [%]                        72.694065
Max Gross Exposure [%]                        117.824629
Max Drawdown [%]                               53.565766
Max Drawdown Duration         25 days 08:39:01.574032903
Total Orders                                 2769.510222
Total Fees Paid                                      0.0
Total Trades                                 1727.124889
Win Rate [%]                                   50.125835
Best Trade [%]                 

In [ ]:
multiple_pf.trades.count()

lmsSlope3_lms_threshold  lmsSlope3_long_slope_thresh  lmsSlope3_short_slope_thresh
0.10839                  -0.00405                     -0.00405                           0
                                                      -0.00324                           0
                                                      -0.00243                           0
                                                      -0.00162                           0
                                                      -0.00081                           0
                                                                                      ... 
1.28250                  0.00324                      -0.00000                        2446
                                                      0.00081                         1995
                                                      0.00162                         1763
                                                      0.00243                         1679
       

In [13]:
num_trade_filter = multiple_pf.trades.count() > 100

# Use these indexes to filter multiple_pf
filtered_pf = multiple_pf.loc[:, num_trade_filter]

In [14]:
filtered_pf.trades.count()

lmsSlope3_lms_threshold  lmsSlope3_long_slope_thresh  lmsSlope3_short_slope_thresh
0.60000                  0.000000                     -0.004049                        831
                                                      -0.003779                        831
                                                      -0.003509                        841
                                                      -0.003239                        862
                                                      -0.002969                        884
                                                                                      ... 
0.99137                  0.003779                      0.002699                       1644
                                                       0.002969                       1651
                                                       0.003239                       1655
                                                       0.003509                       1663
       

In [15]:
filtered_pf.total_return


lmsSlope3_lms_threshold  lmsSlope3_long_slope_thresh  lmsSlope3_short_slope_thresh
0.60000                  0.000000                     -0.004049                      -0.286462
                                                      -0.003779                      -0.286462
                                                      -0.003509                      -0.280287
                                                      -0.003239                      -0.164942
                                                      -0.002969                      -0.121484
                                                                                        ...   
0.99137                  0.003779                      0.002699                      -0.133822
                                                       0.002969                      -0.097382
                                                       0.003239                      -0.129937
                                                       0.00350

In [19]:
metrics = [
    filtered_pf.total_return,
    filtered_pf.trades.win_rate,
    filtered_pf.sharpe_ratio,
    filtered_pf.sortino_ratio,
    filtered_pf.max_drawdown,
    filtered_pf.trades.profit_factor,
    filtered_pf.trades.direction_long.count(),
    filtered_pf.trades.direction_short.count(),
    filtered_pf.trades.direction_long.pnl.sum(),
    filtered_pf.trades.direction_short.pnl.sum()
]

keys = [
    'total_return',
    'win_rate',
    'sharpe_ratio',
    'sortino_ratio',
    'max_drawdown',
    'profit_factor',
    'long_count',
    'short_count',
    'long_pnl_sum',
    'short_pnl_sum'
]

combined_stats = pd.concat(metrics, axis=1, keys=keys)
combined_stats


total_return  \
lmsSlope3_lms_threshold lmsSlope3_long_slope_thresh lmsSlope3_short_slope_thresh                 
0.60000                 0.000000                    -0.004049                        -0.286462   
                                                    -0.003779                        -0.286462   
                                                    -0.003509                        -0.280287   
                                                    -0.003239                        -0.164942   
                                                    -0.002969                        -0.121484   
...                                                                                        ...   
0.99137                 0.003779                     0.002699                        -0.133822   
                                                     0.002969                        -0.097382   
                                                     0.003239                        -0.129937   
                                                     0.003509                        -0.180347   
                                                     0.003779                         0.100446   

                                                                                  win_rate  \
lmsSlope3_lms_threshold lmsSlope3_long_slope_thresh lmsSlope3_short_slope_thresh             
0.60000                 0.000000                    -0.004049                     0.493381   
                                                    -0.003779                     0.493381   
                                                    -0.003509                     0.498216   
                                                    -0.003239                     0.502320   
                                                    -0.002969                     0.503394   
...                                                                                    ...   
0.99137                 0.003779                     0.002699                     0.510341   
                                                     0.002969                     0.503937   
                                                     0.003239                     0.512387   
                                                     0.003509                     0.504510   
                                                     0.003779                     0.512005   

                                                                                  sharpe_ratio  \
lmsSlope3_lms_threshold lmsSlope3_long_slope_thresh lmsSlope3_short_slope_thresh                 
0.60000                 0.000000                    -0.004049                        -1.161807   
                                                    -0.003779                        -1.161807   
                                                    -0.003509                        -1.111983   
                                                    -0.003239                        -0.283536   
                                                    -0.002969                        -0.000038   
...                                                                                        ...   
0.99137                 0.003779                     0.002699                         0.442140   
                                                     0.002969                         0.606429   
                                                     0.003239                         0.465157   
                                                     0.003509                         0.231941   
                                                     0.003779                         1.388238   

                                                                                  sortino_ratio  \
lmsSlope3_lms_threshold lmsSlope3_long_slope_thresh lmsSlope3_short_slope_thresh                  
0.60000                 0.000000                    -0.004049                         -1.664920   
            

Sort by a metric, eg. Total Return

In [29]:
# combined_stats.sort_values(by='win_rate', ascending=False).head(20)
combined_stats.sort_values(by='total_return', ascending=False).head(20)
# combined_stats.sort_values(by='sortino_ratio', ascending=False).head(20)
# combined_stats.sort_values(by='profit_factor', ascending=False).head(20)
# combined_stats.sort_values(by='max_drawdown', ascending=False).head(20)



total_return  \
lmsSlope3_lms_threshold lmsSlope3_long_slope_thresh lmsSlope3_short_slope_thresh                 
0.817428                0.002699                    -0.003509                         2.488958   
                        0.003509                    -0.003509                         2.486723   
0.860913                0.002699                    -0.003509                         2.267672   
0.817428                0.002699                    -0.002969                         2.194164   
                        0.001889                    -0.002969                         2.179215   
                        0.003239                    -0.003509                         2.081134   
0.860913                0.003239                    -0.003509                         2.008456   
0.817428                0.001350                    -0.002969                         2.005794   
                        0.000810                    -0.002969                         1.989430   
                        0.002699                    -0.002699                         1.951541   
                        0.002429                    -0.002429                         1.931017   
0.860913                0.002699                    -0.002699                         1.896288   
0.773942                0.003239                    -0.003509                         1.885470   
0.817428                0.002159                    -0.002969                         1.879471   
                        0.002699                    -0.003239                         1.795332   
                        0.001889                    -0.003509                         1.779587   
                        0.002159                    -0.003509                         1.779401   
0.773942                0.003509                    -0.003509                         1.745613   
0.860913                0.002429                    -0.002429                         1.732236   
                        0.001350                    -0.002699                         1.724070   

                                                                                  win_rate  \
lmsSlope3_lms_threshold lmsSlope3_long_slope_thresh lmsSlope3_short_slope_thresh             
0.817428                0.002699                    -0.003509                     0.520710   
                        0.003509                    -0.003509                     0.563184   
0.860913                0.002699                    -0.003509                     0.503898   
0.817428                0.002699                    -0.002969                     0.525257   
                        0.001889                    -0.002969                     0.515295   
                        0.003239                    -0.003509                     0.534790   
0.860913                0.003239                    -0.003509                     0.522610   
0.817428                0.001350                    -0.002969                     0.515789   
                        0.000810                    -0.002969                     0.509958   
                        0.002699                    -0.002699                     0.519755   
                        0.002429                    -0.002429                     0.521721   
0.860913                0.002699                    -0.002699                     0.504516   
0.773942                0.003239                    -0.003509                     0.539560   
0.817428                0.002159                    -0.002969                     0.514816   
                        0.002699                    -0.003239                     0.515744   
                        0.001889                    -0.003509                     0.509445   
                        0.002159                    -0.003509                     0.516970   
0.773942                0.003509                    -0.003509                     0.554318   
0.860913                0.002429             

In [ ]:
filtered_pf.sortino_ratio.vbt.volume().show()

Testing saving and loading objects

In [ ]:
filtered_pf.total_return.vbt.save('filtered_pf_total_return.pkl')
multiple_pf.total_return.vbt.save('multiple_pf_total_return.pkl')
multiple_pf.total_return.to_csv('multiple_pf_total_return.csv')
filtered_pf.total_return.to_csv('filtered_pf_total_return.csv')
multiple_pf.save('multiple_pf.pkl')
filtered_pf.save('filtered_pf.pkl')

test=vbt.load('multiple_pf_total_return.pkl')
test.volume().show()

In [ ]:
# Plot the winrates
filtered_pf.trades.win_rate.dropna().vbt.volume().show() # Note you have to drop na for the volume to plot

In [ ]:
# Print the best combination of parameters for the LSTM model

# Isolate the best total return portfolio
best_total_return = filtered_pf.total_return.max()
print(f'The best total return of all the combinations is {best_total_return:.2%}')
best_total_return_combination = filtered_pf.total_return.idxmax()
print(f'The best combination for Total Return is {best_total_return_combination}')

# Isolate the best max drawdown
best_max_drawdown = filtered_pf.max_drawdown.max()
print(f'The best max drawdown of all the combinations is {best_max_drawdown:.2%}')
best_max_drawdown_combination = filtered_pf.max_drawdown.idxmax()
print(f'The best combination for max drawdown is {best_max_drawdown_combination}')

# Isolate the best Sharpe ratio portfolio
best_sharpe = filtered_pf.sharpe_ratio.max()
print(f'The best Sharpe ratio of all the combinations is {best_sharpe:.2f}')
best_sharpe_combination = filtered_pf.sharpe_ratio.idxmax()
print(f'The best combination for Sharpe Ratio is {best_sharpe_combination}')

# Isolate the best Win Rate
best_win_rate = filtered_pf.trades.win_rate.max()
print(f'The best Win Rate of all the combinations is {best_win_rate:.2%}')
best_win_rate_combination = filtered_pf.trades.win_rate.idxmax()
print(f'The best combination for Win Rate is {best_win_rate_combination}')

# isolate the best Profit Factor
best_profit_factor = filtered_pf.trades.profit_factor.max()
print(f'The best Profit Factor of all the combinations is {best_profit_factor:.2f}')
best_profit_factor_combination = filtered_pf.trades.profit_factor.idxmax()
print(f'The best combination for Profit Factor is {best_profit_factor_combination}')

In [ ]:
print(multiple_pf[best_total_return_combination].stats())
multiple_pf[best_total_return_combination].plot().show()

In [ ]:
print(multiple_pf[best_win_rate_combination].stats())
multiple_pf[best_win_rate_combination].plot().show()

In [ ]:
print(multiple_pf[best_sharpe_combination].stats())
multiple_pf[best_sharpe_combination].plot().show()

In [ ]:
# Run best profit factor sim

print(multiple_pf[best_profit_factor_combination].stats())
multiple_pf[best_profit_factor_combination].plot(height=900).show()